<a href="https://www.kaggle.com/code/rputtam/pyspark-explore?scriptVersionId=163240785" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## This notebook aims to utilize pyspark and explore various functions for data analysis and manipulation.

Resources for pySpark 
1. https://sparkbyexamples.com/pyspark-tutorial/
2. https://github.com/spark-examples/pyspark-examples
3. https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html?hl=en-GB

##### There are two types of pyspark rdd operations - transformations and actions
* Transformations return rdds
* Actions return the result (non-rdds)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv


## Install PySpark

In [3]:
# Installing Pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=043b9515dd929c24938802e91b8892f4e1492a00822d912e7774c990fbcf9670
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Create SparkSession

In [6]:
# Importing and creating SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('explore_pyspark').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 23:10:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Importing necessary packages
from pyspark.sql.types import *
#from pyspark.sql.functions import *
import pyspark.sql.functions as F

## Loading data

In [7]:
# Reading the kaggle dataset without inferschema
df = spark.read.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv',header=True)
df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: string (nullable = true)
 |-- age: string (nullable = true)
 |-- previous_year_rating: string (nullable = true)
 |-- length_of_service: string (nullable = true)
 |-- KPIs_met_more_than_80: string (nullable = true)
 |-- awards_won: string (nullable = true)
 |-- avg_training_score: string (nullable = true)



In [8]:
# Reading the kaggle dataset with inferschema and checking the data types
df = spark.read.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv',inferSchema= True,header=True)
#inferSchema infers the datatype of field from its values. But it reads the entire data and may have performance issues while reading larger datasets.
#The otherway is to predefine the schema using structType that saves time - Optimized approach
df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met_more_than_80: integer (nullable = true)
 |-- awards_won: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)



In [9]:
#Pre-defining the schmea
# Note, all the fields available in the data need to be defined, the size of the structType and dataframe should match

sch = StructType([
    StructField("employee_id",IntegerType(),True),
    StructField("department",StringType(),True),
    StructField("region",StringType(),True),
    StructField("education",StringType(),True),
    StructField("gender",StringType(),True),
    StructField("recruitment_channel",StringType(),True),
    StructField("no_of_trainings",IntegerType(),True),
    StructField("age",IntegerType(),True),
    StructField("previous_year_rating",IntegerType(),True),
    StructField("length_of_service",IntegerType(),True),
    StructField("KPIs_met_more_than_80",IntegerType(),True),
    StructField("awards_won",IntegerType(),True),
    StructField("avg_training_score",IntegerType(),True)
])

In [83]:
df3 = spark.read\
.option("header",True) \
.schema(sch) \
.csv('/kaggle/input/employees-performance-for-hr-analytics/Uncleaned_employees_final_dataset (1).csv')

df3.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met_more_than_80: integer (nullable = true)
 |-- awards_won: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)



## withColumn()

### .lit()

In [11]:
# Creating new column with constant value
df2 = df.withColumn("company",F.lit('ABC'))\
        .withColumn("country", F.lit('USA'))
df2.select('employee_id','company','country').show(2)

+-----------+-------+-------+
|employee_id|company|country|
+-----------+-------+-------+
|       8724|    ABC|    USA|
|      74430|    ABC|    USA|
+-----------+-------+-------+
only showing top 2 rows



In [12]:
# Changing column datatypes of selected fields using withColumn

df2 = df.withColumn("age",df.age.cast(IntegerType())) \
.withColumn("length_of_service",df.length_of_service.cast(IntegerType()))
df2.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met_more_than_80: integer (nullable = true)
 |-- awards_won: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)



In [13]:
# Rename column
df2 = df2.withColumnsRenamed({'country':'nation','company':'org'})
df2.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- region: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- recruitment_channel: string (nullable = true)
 |-- no_of_trainings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- previous_year_rating: integer (nullable = true)
 |-- length_of_service: integer (nullable = true)
 |-- KPIs_met_more_than_80: integer (nullable = true)
 |-- awards_won: integer (nullable = true)
 |-- avg_training_score: integer (nullable = true)



In [14]:
df2=df2.drop("org","nation")
df2.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met_more_than_80',
 'awards_won',
 'avg_training_score']

## .select()

In [15]:
# selecting few fields using select
x = df3.select('employee_id','age')
print(type(x))
x.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------+---+
|employee_id|age|
+-----------+---+
|       8724| 24|
|      74430| 31|
|      72255| 31|
|      38562| 31|
|      64486| 30|
+-----------+---+
only showing top 5 rows



## .filter()

In [16]:
#filter with one condition
df3.filter(df3.age > 20).select('employee_id','age').show(5)

+-----------+---+
|employee_id|age|
+-----------+---+
|       8724| 24|
|      74430| 31|
|      72255| 31|
|      38562| 31|
|      64486| 30|
+-----------+---+
only showing top 5 rows



In [17]:
#Filter with multiple conditions
df3.filter( (df3.age >20) & (df3.department == "HR") ).select('employee_id',
                                                              'department',
                                                              'age').show(4)

+-----------+----------+---+
|employee_id|department|age|
+-----------+----------+---+
|      74430|        HR| 31|
|      10761|        HR| 58|
|      50038|        HR| 33|
|      50380|        HR| 31|
+-----------+----------+---+
only showing top 4 rows



In [18]:
# Finance and age>40 or HR 
df4 = df3.filter( ( (df3.department == "Finance") & (df3.age>40) ) | (df3.department == "HR") )\
        .select('employee_id','department','age')

In [68]:
df4.groupBy('department')\
.count()\
.show()

+----------+-----+
|department|count|
+----------+-----+
|        HR|  833|
|   Finance|   78|
+----------+-----+



In [19]:
df4.filter((df4.department=="Finance") & (df4.age<40)).show()

+-----------+----------+---+
|employee_id|department|age|
+-----------+----------+---+
+-----------+----------+---+



### Get unique values for categorical fields

In [20]:
df3.select('department')\
.distinct().show()

+-----------------+
|       department|
+-----------------+
|               HR|
|          Finance|
|        Analytics|
|            Legal|
|Sales & Marketing|
|       Technology|
|      Procurement|
|       Operations|
|              R&D|
+-----------------+



In [21]:
# to get unique values as a list
df3.agg(F.collect_set("department")).collect()[0][0]

['Finance',
 'Legal',
 'Sales & Marketing',
 'HR',
 'Operations',
 'R&D',
 'Analytics',
 'Technology',
 'Procurement']

In [22]:
df3.agg(F.collect_set("recruitment_channel")).collect()[0][0]

['other', 'referred', 'sourcing']

In [23]:
dept =['R&D','Technology','Analytics']
df4=df3.filter(df3.department.isin(dept))
df4.select(df4.department).distinct().show()

+----------+
|department|
+----------+
| Analytics|
|Technology|
|       R&D|
+----------+



In [24]:
df3.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met_more_than_80',
 'awards_won',
 'avg_training_score']

## Aggregate functions

In [25]:
df3.groupBy(['department'])\
   .agg(F.round(F.median('age'),2).alias('median_age'),
       F.mean('avg_training_score').alias('avg_training_score'),
        F.mode('length_of_service').alias('max_los')
       ).show()

+-----------------+----------+------------------+-------+
|       department|median_age|avg_training_score|max_los|
+-----------------+----------+------------------+-------+
|               HR|      32.0| 50.38775510204081|      2|
|          Finance|      31.0| 60.32668329177057|      2|
|        Analytics|      31.0| 84.56511490866235|      4|
|            Legal|      32.0| 59.53313253012048|      2|
|Sales & Marketing|      33.0| 50.05661414437523|      3|
|       Technology|      33.0| 79.84765802637563|      4|
|      Procurement|      34.5| 70.18035714285715|      4|
|       Operations|      35.0|60.351589103291715|      4|
|              R&D|      32.0| 84.45180722891567|      2|
+-----------------+----------+------------------+-------+



In [26]:
df3.groupBy(['recruitment_channel'])\
   .agg(F.median('age').alias('median_age'),
       F.mean('no_of_trainings').alias('trainings_avg'))\
   .show()

+-------------------+----------+------------------+
|recruitment_channel|median_age|     trainings_avg|
+-------------------+----------+------------------+
|              other|      33.0|1.2548456568557071|
|           sourcing|      33.0|1.2484691794802014|
|           referred|      31.0|1.1766561514195584|
+-------------------+----------+------------------+



## .explode()

In [27]:
#Let's create a new column-languages known for each employee
# For simplicity, creating same values for all employees

df3=df3.withColumn('languages',F.lit(['SQL','Python']))
df3.select('employee_id','languages').show(5)

+-----------+-------------+
|employee_id|    languages|
+-----------+-------------+
|       8724|[SQL, Python]|
|      74430|[SQL, Python]|
|      72255|[SQL, Python]|
|      38562|[SQL, Python]|
|      64486|[SQL, Python]|
+-----------+-------------+
only showing top 5 rows



In [29]:
df_lang = df3.select(df3.employee_id,F.explode(df3.languages))
df_lang.show(5)

+-----------+------+
|employee_id|   col|
+-----------+------+
|       8724|   SQL|
|       8724|Python|
|      74430|   SQL|
|      74430|Python|
|      72255|   SQL|
+-----------+------+
only showing top 5 rows



##### explode can also be used to breakdown a column in dictionary format similar to list. This creates two new columns - key and value

## udf function

##### UDF is User Defined Function that can be used to modify columns or fields

In [86]:
#udf on one column

# Define function
def cap_gender(g):
    return g.upper()

# udf creation
gen_udf = F.udf(cap_gender,StringType())

# Apply udf
df_gen = df3.withColumn('gender_upd',gen_udf(df3.gender))

In [89]:
df_gen.select('gender','gender_upd').show(3)

+------+----------+
|gender|gender_upd|
+------+----------+
|     m|         M|
|     f|         F|
|     m|         M|
+------+----------+
only showing top 3 rows



##### UDF with multiple returntypes

In [64]:
def modify_dataframe(department,recruitment_channel):
    mod_dept = department.upper()
    mod_rec_channel = recruitment_channel.title()
    return mod_dept,mod_rec_channel

In [65]:
#Register UDF
df_udf = F.udf(modify_dataframe,StructType([StructField('mod_dept',StringType()),
                                           StructField('mode_rec_channel',StringType())]))

In [66]:
#Apply udf to dataframe
df4 = df3.withColumn('mod_columns',df_udf(df3['department'],df3['recruitment_channel']))

In [93]:
df4.select('mod_columns').show(4)

+--------------------+
|         mod_columns|
+--------------------+
|{TECHNOLOGY, Sour...|
|         {HR, Other}|
|{SALES & MARKETIN...|
|{PROCUREMENT, Other}|
+--------------------+
only showing top 4 rows



In [92]:
#Split the udf updated columns
df5=df4.select('mod_columns.*').drop('mod_columns')
df5.show(4)

+-----------------+----------------+
|         mod_dept|mode_rec_channel|
+-----------------+----------------+
|       TECHNOLOGY|        Sourcing|
|               HR|           Other|
|SALES & MARKETING|           Other|
|      PROCUREMENT|           Other|
+-----------------+----------------+
only showing top 4 rows

